**Problem Statement**

This is a problem to identify wheather a given news is fake or real.
The Indepedent variables or factors used here can be title of news or news lines themselves.
We will be looking at different model to solve this problem.
In this notebook we will using Bag of Words model.

**Dataset**

The dataset for news is taken from Kaggle https://www.kaggle.com/c/fake-news/.
The dataset is quite huge containing thousands of rows, we expect our model to be highly accurate.

# Step 1 - Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Step 2 - Importing the dataset

In [2]:
data = pd.read_csv('/content/drive/My Drive/Data Science Projects/Dataset Fakenews.csv') # 1 is for fake 0 for real
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


# Step 3 - Dealing with null values

In [3]:
print(data.isnull().sum())
data.dropna(inplace = True)
data.reset_index(inplace = True)

id           0
title      558
author    1957
text        39
label        0
dtype: int64


In [4]:
print(data.isnull().sum())

index     0
id        0
title     0
author    0
text      0
label     0
dtype: int64


# Step 4 - Making dependent variable Y

In [7]:
Y = data.iloc[:, -1]
# We will be dealing with Independent Variable X later on

In [8]:
Y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

# Step 5 - Cleaning the text

In [9]:
corpus = []                                           # Declaring a list
for i in range(0,data.shape[0]):
  review = re.sub('[^a-zA-Z]', ' ',  data['title'][i])  # Keeping only a-z or A-Z characters in news
  review = review.lower()                               # Lower all characters
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')            # Removing all stopwords/unnecessary words from news
  all_stopwords.remove('not') #otherwise this word "not" will be included in stopwords removing of which from news is not idle
  review = [ps.stem(word) for word in review if not word in set(all_stopwords) ]
  review = ' '.join(review) #Converting back the review list to String
  corpus.append(review)

# Step 6 - Creating Bag of Words

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, ngram_range = (1,3)) #take feature as combination of words 
X = cv.fit_transform(corpus).toarray()  # Finally independent variable X is ready

# Step 7 - Splitting dataset into Training and Test

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X, Y, test_size = 0.20, random_state = 0)

# Step 8 - Creating Machine Learning models

In [21]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [23]:

from sklearn.linear_model import PassiveAggressiveClassifier
classifier = PassiveAggressiveClassifier()
classifier.fit(X_train, Y_train)


PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='hinge', max_iter=1000, n_iter_no_change=5,
                            n_jobs=None, random_state=None, shuffle=True,
                            tol=0.001, validation_fraction=0.1, verbose=0,
                            warm_start=False)

# Step 9 - Predicting results and checking accuracy

In [24]:
yhat = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(Y_test, yhat)
accuracy = accuracy_score(Y_test, yhat)

print(accuracy)
print(cm)

0.9212469237079574
[[1875  165]
 [ 123 1494]]


# Step 10 - Checking which wards are most real and fake

In [18]:
feature_names = cv.get_feature_names()
coefficients = classifier.coef_[0]

# Top 20 real news words
sorted(zip(coefficients, feature_names), reverse = True)[:20]

[(3.8571537425702975, 'journal'),
 (3.7815966491552446, 'idiot'),
 (3.775170399164279, 'invad'),
 (3.581588917914471, 'gap'),
 (3.5745012060691366, 'trump need'),
 (3.429834890494482, 'american peopl'),
 (3.3814146979843276, 'humili'),
 (3.3047458357515245, 'hillari'),
 (3.301384952919329, 'poll show'),
 (3.1881157247178455, 'comment'),
 (3.1085406386443104, 'daesh'),
 (3.093216287037252, 'negoti'),
 (3.018388392428779, 'migrant crisi'),
 (2.893182234438597, 'report new york'),
 (2.862984274145243, 'jame matti'),
 (2.8318501875211615, 'quak'),
 (2.745629786078349, 'report new'),
 (2.687209547996219, 'gingrich'),
 (2.6783990977277856, 'trap'),
 (2.665046060322914, 'meddl')]

In [19]:
# Top 20 fake news words
sorted(zip(coefficients, feature_names), reverse = False)[:20]

[(-7.271551256920901, 'breitbart'),
 (-4.3353405388308195, 'hillari clinton'),
 (-3.7972721545682884, 'penc'),
 (-3.7685837407901874, 'clinton aid'),
 (-3.7575113708312093, 'ross'),
 (-3.729416320091774, 'todd'),
 (-3.7248559551882026, 'virgil'),
 (-3.6953307726288314, 'delingpol'),
 (-3.5911267754762535, 'espn'),
 (-3.5212977714349507, 'streisand'),
 (-3.4618303179314287, 'cher'),
 (-3.442789691327568, 'macron'),
 (-3.377984925443203, 'town hall'),
 (-3.319626386785692, 'ck'),
 (-3.3068067021872736, 'abort'),
 (-3.2976365736937137, 'matti'),
 (-3.2208307687016626, 'gorka'),
 (-3.2026418384766835, 'new year'),
 (-3.2002125250061666, 'regist'),
 (-3.199840168963801, 'trump order')]

# Step 11 - Saving model for future use and deplyoment

In [20]:
import pickle 

with open("model.pkl", "wb") as filename:
  pickle.dump(classifier, filename)

with open("cv.pkl" , "wb") as filename:
  pickle.dump(cv, filename)

**Conclusion**

Lets compare accuracies by different models

**Multinomial NB**

Accuracy - 0.9004648619086683

**Passive Agressive Classifier**

0.9212469237079574

Thus, it can be clearly seen that both the models Multinomial NB and Passive Agressive Classifier are great for NLP classification problems.
Since Passive Agressive gave us a bit more accuracy , we will be chosing it for this specific problem.

